In [ ]:
# | default_exp _components.new_project_generator

In [ ]:
# | export


from typing import *
import shutil
from pathlib import Path
import os
from tempfile import TemporaryDirectory

from yaspin import yaspin

from faststream_gen._components.logger import get_logger
from faststream_gen._code_generator.helper import (
    download_and_extract_faststream_archive,
    write_file_contents,
    read_file_contents,
    CustomAIChat,
    ValidateAndFixResponse
)

from faststream_gen._code_generator.constants import (
    FASTSTREAM_TEMPLATE_ZIP_URL,
    FASTSTREAM_TEMPLATE_DIR_SUFFIX,
    INTERMEDIATE_RESULTS_DIR_NAME,
    APPLICATION_FILE_NAME,
    INTEGRATION_TEST_FILE_NAME
)

from faststream_gen._code_generator.prompts import REQUIREMENTS_GENERATION_PROMPT

In [ ]:


import pytest

from faststream_gen._components.logger import suppress_timestamps
from faststream_gen._code_generator.constants import DESCRIPTION_FILE_NAME, OpenAIModel

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export

def _split_requirements(response: str) -> Tuple[str, str]:
    app_code, test_code = response.split("### requirements.txt ###")[1].split("### dev_requirements.txt ###")
    return app_code, test_code

def _validate_response(response: str) -> List[str]:
    try:
        requirements, dev_requirements = _split_requirements(response)
        return []
    except (IndexError, ValueError) as e:
        return [
            "Please add ### requirements.txt ### and ### dev_requirements.txt ### in your response"
        ]

In [ ]:
fixture_response = """
### requirements.txt ###
pandas

### dev_requirements.txt ###
pytest
"""

expected = []
actual = _validate_response(fixture_response)
print(actual)
assert actual == expected

[]


In [ ]:
fixture_response = """
### requirements.txt ###
pandas

### dev_requirements.txt ##
pytest
"""

expected = ['Please add ### requirements.txt ### and ### dev_requirements.txt ### in your response']
actual = _validate_response(fixture_response)
print(actual)
assert actual == expected

['Please add ### requirements.txt ### and ### dev_requirements.txt ### in your response']


In [ ]:
# | export


def _generate_requirements(
    d: str, model: str, total_usage: List[Dict[str, int]]
) -> Tuple[str, str, List[Dict[str, int]]]:
    
    app_code = read_file_contents(f"{d}/app/application.py")
    requirements = read_file_contents(f"{d}/requirements.txt")
    dev_requirements = read_file_contents(f"{d}/dev_requirements.txt")

    prompt = (
        REQUIREMENTS_GENERATION_PROMPT
        + app_code
        + "\n==== REQUIREMENT ====\n"
        + requirements
        + "\n==== DEV REQUIREMENT ====\n"
        + dev_requirements
    )
    requirements_generator = CustomAIChat(
        model=model,
        user_prompt=prompt,
    )
    requirements_validator = ValidateAndFixResponse(
        requirements_generator, _validate_response
    )
    requirements, total_usage = requirements_validator.fix(
        prompt,
        total_usage=total_usage,
    )

    requirements, dev_requirements = _split_requirements(
        requirements
    )

    return requirements, dev_requirements, total_usage

In [ ]:
# | notest

fixture = """
from faststream import FastStream
from faststream.rabbit import RabbitBroker

broker = RabbitBroker()
app = FastStream(broker)


@broker.subscriber("routing_key")  # handle messages by routing key
async def handle(msg):
    print(msg)


@app.after_startup
async def test_publish():
    await broker.publish(
        "message",
        "routing_key",  # publish message with routing key
    )
"""

with TemporaryDirectory() as d:
    app_code = write_file_contents(f"{d}/app/application.py", fixture)
    requirements = write_file_contents(
        f"{d}/requirements.txt", "\nfaststream[docs]==0.0.1.dev20230912"
    )
    dev_requirements = write_file_contents(
        f"{d}/dev_requirements.txt", "\nfaststream[testing]==0.0.1.dev20230912"
    )

    requirements, dev_requirements, total_usage = _generate_requirements(d, OpenAIModel.gpt4.value, [])

    print(requirements)
    assert requirements.replace("\n", "") == "faststream[rabbit, docs]==0.0.1.dev20230912"
    
    print(dev_requirements)
    assert dev_requirements.replace("\n", "") == "faststream[rabbit, testing]==0.0.1.dev20230912"



faststream[rabbit, docs]==0.0.1.dev20230912




faststream[rabbit, testing]==0.0.1.dev20230912


In [ ]:
# | export


def create_project(
    output_path: str,
    save_intermediate_files: bool,
    model: str,
    total_usage: List[Dict[str, int]],
) -> List[Dict[str, int]]:
    with yaspin(
        text="Creating a new FastStream project...", color="cyan", spinner="clock"
    ) as sp:
        with download_and_extract_faststream_archive(
            FASTSTREAM_TEMPLATE_ZIP_URL
        ) as extracted_path:
            with TemporaryDirectory() as tmp_dir:
                app_path = f"{tmp_dir}/app/application.py"
                test_path = f"{tmp_dir}/tests/test_application.py"

                intermediate_dir_path = f"{output_path}/{INTERMEDIATE_RESULTS_DIR_NAME}"
                shutil.copytree(
                    str(extracted_path / FASTSTREAM_TEMPLATE_DIR_SUFFIX),
                    tmp_dir,
                    dirs_exist_ok=True,
                )
                shutil.copy(
                    f"{intermediate_dir_path}/{APPLICATION_FILE_NAME}", app_path
                )
                shutil.copy(
                    f"{intermediate_dir_path}/{INTEGRATION_TEST_FILE_NAME}", test_path
                )

                test_file_contents = read_file_contents(test_path)
                test_file_contents = test_file_contents.replace(
                    "from application import", "from app.application import"
                )
                write_file_contents(test_path, test_file_contents)

                requirements, dev_requirements, total_usage = _generate_requirements(tmp_dir, model, total_usage)

                requirements_file = f"{tmp_dir}/requirements.txt"
                write_file_contents(requirements_file, requirements)

                dev_requirements_file = f"{tmp_dir}/dev_requirements.txt"
                write_file_contents(dev_requirements_file, dev_requirements)

                shutil.copytree(tmp_dir, output_path, dirs_exist_ok=True)
                if not save_intermediate_files:
                    shutil.rmtree(intermediate_dir_path)

        sp.text = ""
        sp.ok(f" ✔ New FastStream project created.")
        return total_usage

In [ ]:
# | notest

fixture_application_code = """
message = "hi"
print(message)
"""

fixture_test_code = """
from application import message
"""

fixture_description = """
description
"""

for flag in [True, False]:
    with TemporaryDirectory() as d:
        intermediate_results_dir = Path(d)/INTERMEDIATE_RESULTS_DIR_NAME
        intermediate_results_dir.mkdir(parents=True, exist_ok=True)

        app_file_name = intermediate_results_dir / APPLICATION_FILE_NAME
        write_file_contents(str(app_file_name), fixture_application_code)

        test_file_name = intermediate_results_dir / INTEGRATION_TEST_FILE_NAME
        write_file_contents(str(test_file_name), fixture_test_code)

        description_file_name = intermediate_results_dir / DESCRIPTION_FILE_NAME
        write_file_contents(str(description_file_name), fixture_description)

        create_project(d, flag, OpenAIModel.gpt4.value, [])
        files = [p.stem for p in list(Path(f"{d}").glob("*"))]
        print(files)
        if flag:
            assert INTERMEDIATE_RESULTS_DIR_NAME in files
        else:
            assert INTERMEDIATE_RESULTS_DIR_NAME not in files
        assert "README" in files

 ✔ New FastStream project created.     
['README', '.github', 'dev_requirements', 'LICENSE', '_faststream_gen_tmp', 'tests', 'app', 'requirements', '.gitignore']
 ✔ New FastStream project created.     
['README', '.github', 'dev_requirements', 'LICENSE', 'tests', 'app', 'requirements', '.gitignore']
